#### *Situação Problema* 

##### Uma operadora de telecomunicações está recebendo reclamações de clientes sobre mensagens de spam em seus celulares. Essas mensagens não só irritam os usuários, mas também podem levar a golpes financeiros. A empresa contratou sua equipe para desenvolver um modelo de classificação automática que identifique mensagens spam com alta precisão, evitando bloquear mensagens legítimas (especialmente alertas importantes, como notificações bancárias ou emergenciais).

##### Objetivo: Criar um modelo de machine learning que classifique SMS como ham ou spam com base no conteúdo textual.

##### 1. Carregamento e exploração inicial

In [1]:
import pandas as pd

df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])

df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   label           5572 non-null   object
 1   message         5572 non-null   object
 2   mensagem_limpa  5572 non-null   object
dtypes: object(3)
memory usage: 130.7+ KB



##### Ham vs. Spam

##### Ham = mensagens normais e úteis

##### Spam = mensagens indesejadas, como propagandas, golpes ou notificações invasivas.



In [3]:
# Contar quantas mensagens são ham e quantas são spam

df['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

In [5]:
# Verificar exemplos de mensagens de cada classe (5)

# Exemplo mensagens 'ham' 
print("Mensagens HAM:")
print(df[df['label'] == 'ham']['message'].head(), "\n")

# Exemplo mensagens 'Spam'
print("Mensagens SPAM:")
print(df[df['label'] == 'spam']['message'].head())




Mensagens HAM:
0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
6    Even my brother is not like to speak with me. ...
Name: message, dtype: object 

Mensagens SPAM:
2     Free entry in 2 a wkly comp to win FA Cup fina...
5     FreeMsg Hey there darling it's been 3 week's n...
8     WINNER!! As a valued network customer you have...
9     Had your mobile 11 months or more? U R entitle...
11    SIX chances to win CASH! From 100 to 20,000 po...
Name: message, dtype: object


##### 2. Pré-processamento de texto


In [13]:
# Limpar o texto: remover caracteres especiais, números e converter para minúsculas.

import re # ferramentas para procurar e manipular textos com padrões.

def limpar_texto(texto):
    texto = texto.lower()  # converte para minúsculas
    texto = re.sub(r'[^a-z\s]', '', texto)  # remove números e caracteres especiais
    texto = re.sub(r'\s+', ' ', texto).strip()  # remove espaços duplicados e espaços no início/fim
    return texto

# Aplicar a função de limpeza à coluna 'message'
df['mensagem_limpa'] = df['message'].apply(limpar_texto)

df[['message', 'mensagem_limpa']].head()

,message,mensagem_limpa
0,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in a wkly comp to win fa cup final ...
3,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...


In [14]:
##### Tokenizar as mensagens (dividir o texto em pedaços menores)

df['tokens'] = df['mensagem_limpa'].apply(lambda x: x.split())

# Exibir as colunas: original, limpa e tokens
df[['mensagem_limpa', 'tokens']].head()

,mensagem_limpa,tokens
0,go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o..."
1,ok lar joking wif u oni,"[ok, lar, joking, wif, u, oni]"
2,free entry in a wkly comp to win fa cup final ...,"[free, entry, in, a, wkly, comp, to, win, fa, ..."
3,u dun say so early hor u c already then say,"[u, dun, say, so, early, hor, u, c, already, t..."
4,nah i dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l..."


In [18]:
!pip install nltk

In [19]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

df['tokens'] = df['mensagem_limpa'].apply(word_tokenize)
df[['mensagem_limpa', 'tokens']].head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\debor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,mensagem_limpa,tokens
0,go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o..."
1,ok lar joking wif u oni,"[ok, lar, joking, wif, u, oni]"
2,free entry in a wkly comp to win fa cup final ...,"[free, entry, in, a, wkly, comp, to, win, fa, ..."
3,u dun say so early hor u c already then say,"[u, dun, say, so, early, hor, u, c, already, t..."
4,nah i dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l..."


In [20]:
# Baixar as stopwords 

import nltk
nltk.download('stopwords')  # Baixa lista de stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\debor\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [22]:
from nltk.corpus import stopwords

# Stopwords em inglês
stopwords_en = set(stopwords.words('english'))

In [24]:
# Função para filtrar as stopwords de cada lista de tokens
def remover_stopwords(tokens):
    return [palavra for palavra in tokens if palavra not in stopwords_en]

# Aplicar a função
df['tokens_sem_stopwords'] = df['tokens'].apply(remover_stopwords)

# Visualizar o resultado
df[['mensagem_limpa', 'tokens', 'tokens_sem_stopwords']].head()


,mensagem_limpa,tokens,tokens_sem_stopwords
0,go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n..."
1,ok lar joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,free entry in a wkly comp to win fa cup final ...,"[free, entry, in, a, wkly, comp, to, win, fa, ...","[free, entry, wkly, comp, win, fa, cup, final,..."
3,u dun say so early hor u c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]"
4,nah i dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t..."


##### 3. Vetorização dos Dados

##### Converter texto em features numéricas usando TF-IDF ou Bag of Words.


##### Bag of Words é mais simples, mas pode dar pesos iguais a todas as palavras, mesmo aquelas que não são tão informativas.

##### TF-IDF geralmente funciona melhor em tarefas de classificação de texto, pois ele destaca as palavras mais relevantes para distinguir entre as classes.

* #####  Considerando que nosso objetivo é classificar mensagens como spam ou ham, o TF-IDF tende a ser uma escolha mais eficaz.
  

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializar o vetorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Ajustar o vetorizador aos tokens (para criar o vocabulário)
tfidf_vectorizer.fit(df['tokens_sem_stopwords'].apply(lambda x: ' '.join(x)))

# Transformar os tokens em vetores TF-IDF
tfidf_vectors = tfidf_vectorizer.transform(df['tokens_sem_stopwords'].apply(lambda x: ' '.join(x)))

print("Formato da matriz TF-IDF:", tfidf_vectors.shape)

Formato da matriz TF-IDF: (5572, 8476)


##### 4. Divisão dos Dados


In [34]:
# Separar o dataset em conjuntos de treino (70%) e teste (30%) 

# Features 
X = tfidf_vectors

# Rótulos
y = df['label']

# Divisão em treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Visualizar distribuição original das classes no treino
print("Distribuição original no treino:")
print(pd.Series(y_train).value_counts())

# Aplicando SMOTE para balancear os dados no conjunto de treino
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Visualizar distribuição após o balanceamento
print("\nDistribuição após SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

Distribuição original no treino:
label
ham     3377
spam     523
Name: count, dtype: int64

Distribuição após SMOTE:
label
spam    3377
ham     3377
Name: count, dtype: int64
